In [45]:
import pandas as pd
import numpy as np

In [67]:
df = pd.read_csv('diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [69]:
df['BloodPressure'].min()

0

In [70]:
# it appears that some women have a blood pressure of 0 which is obviusly impossible  therefore it must be a null
# lets get rid of them

In [71]:
blood_pressure_exist = df[df['BloodPressure']!=0]
blood_pressure_exist =blood_pressure_exist['BloodPressure']
blood_pressure_exist

0      72
1      66
2      64
3      66
4      40
       ..
763    76
764    70
765    72
766    60
767    70
Name: BloodPressure, Length: 733, dtype: int64

In [72]:
# I looked it up and diastolic pressure(what we have) above 80 is considered high blood pressure
# so lets make a column of high blood pressure.

In [73]:
df['High_BloodPressure']=df['BloodPressure'].apply(lambda x:1 if x>=80 else 0)
df['High_BloodPressure'].value_counts()


0    563
1    205
Name: High_BloodPressure, dtype: int64

In [74]:
# Lets get rid of bmi =0 cuz thats obviously null

In [75]:
bmi_noNull=df[df['BMI']!=0]
bmi_noNull=bmi_noNull['BMI']
bmi_noNull_avg=bmi_noNull.sum()/len(bmi_noNull)
bmi_noNull_avg=bmi_noNull_avg
df['BMI']=df['BMI'].apply(lambda x:bmi_noNull_avg if x==0 else x)
df['BMI']

0      33.6
1      26.6
2      23.3
3      28.1
4      43.1
       ... 
763    32.9
764    36.8
765    26.2
766    30.1
767    30.4
Name: BMI, Length: 768, dtype: float64

In [ ]:
# I also found that a bmi between 25-30(not including 30) is overweight and 30+ is obese and under 18.5 is underweight so lets
# make a column.

In [78]:
def find_weight(position):
    if position<18.5:
        return 'UnderWeight'
    elif position>=18.5 and position<30:
        return 'Healthy'
    else :
        return 'Obese'
    

In [79]:
df['Wieght']=df['BMI'].apply(find_weight)
df['Wieght'].value_counts()

Obese          483
Healthy        281
UnderWeight      4
Name: Wieght, dtype: int64

In [35]:
df['Obese']=df['BMI'].apply(lambda x:1 if x>=30 else 0)
df['Obese'].value_counts()

1    472
0    296
Name: Obese, dtype: int64

In [44]:
# Medicly anyone age 65 and above is considered elderly so lets make another column

In [43]:
df['Elderly']=df['Age'].apply(lambda x:1 if x>=65 else 0)
df['Elderly']

0      0
1      0
2      0
3      0
4      0
      ..
763    0
764    0
765    0
766    0
767    0
Name: Elderly, Length: 768, dtype: int64

In [ ]:
#Normal blood glucose level (tested while fasting) for non-diabetics is between 3.9 and 7.1 mmol/L (70 and 130 mg/dL)

In [83]:
withzero=df['Glucose']==0
withzero.value_counts()

False    763
True       5
Name: Glucose, dtype: int64

In [88]:
def define_glucose_level(position):
    if position==0:
        return 'Unknown'
    elif position<70:
        return 'low_glucose_levels'
    elif position>=130:
        return 'high_glucose_levels'
    else:
        return 'normal_glucose_levels'
    

In [89]:
df['Glucose_Level']=df['Glucose'].apply(define_glucose_level)
df['Glucose_Level'].value_counts()

normal_glucose_levels    494
high_glucose_levels      258
low_glucose_levels        11
Unknown                    5
Name: Glucose_Level, dtype: int64

In [90]:
a=df['Insulin']==0
a.value_counts()

False    394
True     374
Name: Insulin, dtype: int64